# Prueba Project Manager Finanzas

Usted es Project Manager de Finanzas y se encuentra liderando un proyecto para analizar la situación financiera de la industria bancaria chilena. Dispone de información que ha sido obtenida de la CMF correspondiente al cierre de Abril de 2023, y se le ha encomendado generar un reporte que describa el balance de los 5 principales bancos del país (Santander, Banco de Chile, Banco BCI, Banco Estado, Scotiabank).

## Objetivo del Desafío:

1. Generar un proceso Python para extraer la información de los archivos `b1202304xxx.txt`, donde `b1` tipifica al archivo como balance de situación, luego `202304` indica el período de la información, y los últimos 3 dígitos (marcados como `xxx`) corresponden al id de la institución financiera.

2. Generar un reporte Excel con el resultado del proceso Python. Ver template adjunto.

    ### 2.1. Hoja data:
    
    En esta hoja debe consolidar la extracción de la información de los 5 bancos bajo análisis. Cada línea registrada debe seguir el siguiente formato.
    
    | Periodo | Institución Financiera | Cuenta contable | Saldo CLP | Saldo UF | Saldo USD | Saldo otras monedas |
    
    ### 2.2. Hoja reporte balance:

    En esta hoja, En esta hoja debe consolidar el total de los saldos según cuenta contable. Para lo cual en la celda C4 se debe ingresar el nombre de la institución financiera, de forma tal que automáticamente se actualice la columna saldo del reporte. Ver template adjunto.


In [1]:
#libraries
import pandas as pd
import numpy as np

In [2]:
bancos = [
    'b1202304001', #Banco de Chile
    'b1202304037', #Santander
    'b1202304016', #Estado
    'b1202304012', #BCI
    'b1202304014'  #Scotiabank
]

In [3]:
dfs = {} # Diccionario de dataframes
df_names = {0:'Cuenta_contable', 1:'Saldo_CLP',2:'Saldo_UF',3:'Saldo_USD',4:'Saldo_otras_monedas', 'Archivo':'Archivo', 'Banco':'Banco'}

for banco in bancos:
    ruta = f'./PruebaPM/{banco}.txt' # Ruta del archivo CSV
    df = pd.read_csv(ruta, sep="\t", header=None, names=range(5), encoding='latin1') # Leer el archivo CSV sin el header
    df['Archivo'] = banco
    df['Banco'] = df.iloc[0,0]
    df = df.iloc[1:, :] # Eliminar la primera fila

    dfs[banco] = df # Agregar el dataframe al diccionario

df_final = pd.concat(dfs.values(), ignore_index=True) # Concatenar los dataframes
df_final.rename(columns=df_names, inplace=True)

df_final = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_final.iloc[:, :4] = df_final.iloc[:, :4].replace(',', '.', regex=True).astype(float)
df_final.iloc[:, :4] = df_final.iloc[:, :4].astype(float)


In [4]:
df_final['Periodo'] = df_final['Archivo'].str.extract(r'b1(\d{6})') #Extrae el periodo de la fecha de archivo
df_final['Banco'] = df_final['Banco'].str.extract(r'\b (.*)') # Extrae todo despues del primer espacio

df_final['Periodo'] = pd.to_datetime(df_final['Periodo'], format='%Y%m')# convertir periodo a tipo fecha
df_final['Periodo'] = df_final['Periodo'].dt.strftime('%Y-%m')

df_final = df_final.drop(df_final.columns[5], axis=1)# quitar columna Archivo

In [5]:
df_final.head()

,Cuenta_contable,Saldo_CLP,Saldo_UF,Saldo_USD,Saldo_otras_monedas,Banco,Periodo
0,100000000.0,28444728250232.0,20128453882100.0,117300798857.0,"6694297877237,00",BANCO DE CHILE,2023-04
1,105000000.0,1059628254454.0,0.0,0.0,"2016638137886,00",BANCO DE CHILE,2023-04
2,105000100.0,823187780235.0,0.0,0.0,"0059105027666,00",BANCO DE CHILE,2023-04
3,105000101.0,192860234135.0,0.0,0.0,"0059105027666,00",BANCO DE CHILE,2023-04
4,105000102.0,630327546100.0,0.0,0.0,"0000000000000,00",BANCO DE CHILE,2023-04


In [6]:
df_final.to_csv('resutaldo.csv', sep=';', index=False)